In [1312]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from __future__ import print_function
from itertools import repeat

from datetime import date
import datetime
#imports

In [1313]:
batters_df = pd.read_csv('../../data/individual_batting_stats.csv')
pitchers_df = pd.read_csv('../../data/individual_pitching_stats.csv')
pitcher_run_value = pd.read_csv('../../data/pitch-arsenal-stats-pitchers.csv')
batter_run_value = pd.read_csv('../../data/pitch-arsenal-stats-batters.csv')

In [1314]:
batter_run_value.sort_values(by=['run_value_per_100'])

,last_name,first_name,player_id,team_name_alt,pitch_type,pitch_name,run_value_per_100,run_value,pitches,pitch_usage,...,ba,slg,woba,whiff_percent,k_percent,put_away,est_ba,est_slg,est_woba,hard_hit_percent
1516,Barrera,Luis,642456,OAK,CH,Changeup,-10.2,-5,46,14.6,...,0.091,0.091,0.082,12.9,13.6,18.8,0.190,0.271,0.195,31.6
2308,Mazeika,Patrick,664850,NYM,SIFT,Sinker,-10.2,-2,23,12.3,...,0.000,0.000,0.000,16.7,0.0,0.0,0.196,0.420,0.253,20.0
2343,Ahmed,Nick,605113,ARI,CH,Changeup,-9.9,-2,24,11.4,...,0.100,0.200,0.125,30.8,40.0,44.4,0.145,0.197,0.145,33.3
2301,Simmons,Andrelton,592743,CHC,CH,Changeup,-9.2,-3,31,11.5,...,0.000,0.000,0.000,41.2,20.0,18.2,0.091,0.096,0.083,12.5
2306,Upton,Justin,457708,SEA,SIFT,Sinker,-9.1,-3,36,19.7,...,0.000,0.000,0.140,25.0,20.0,33.3,0.136,0.247,0.269,66.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,Riley,Austin,663586,ATL,CUKC,Curveball,9.8,10,104,8.0,...,0.441,0.912,0.566,43.9,32.4,25.0,0.333,0.558,0.374,65.2
1860,Kirilloff,Alex,666135,MIN,SIFT,Sinker,10.1,5,52,13.5,...,0.467,0.800,0.606,0.0,0.0,0.0,0.367,0.791,0.492,66.7
2341,Lindor,Francisco,596019,NYM,FS,Splitter,10.2,2,22,1.5,...,0.222,0.333,0.305,26.7,10.0,8.3,0.242,0.261,0.218,33.3
1248,Naquin,Tyler,571980,CIN,CH,Changeup,10.5,7,65,11.8,...,0.308,0.615,0.402,23.3,15.4,16.0,0.320,0.633,0.397,45.5


# Getting starting lineups and pitchers

In [1315]:
url = 'https://www.mlb.com/starting-lineups/2022-07-17'

In [1316]:
res = requests.get(url)
soup = BeautifulSoup(res.content)

In [1317]:
away_team = soup.find_all('div', {'class': 'starting-lineups__teams--away-head'})
home_team = soup.find_all('div', {'class': 'starting-lineups__teams--home-head'})
home_lineup = soup.find_all('ol', {'class': 'starting-lineups__team starting-lineups__team--away'})
away_lineup = soup.find_all('ol', {'class': 'starting-lineups__team starting-lineups__team--home'})

In [1318]:
away_teams = []
for i in away_team:
    away_teams.append(str(i))
home_teams = []
for i in home_team:
    home_teams.append(str(i))
home_lineups = []
for i in home_lineup:
    home_lineups.append(str(i))
away_lineups = []
for i in away_lineup:
    away_lineups.append(str(i))
#Getting scraped data into strings

The list has duplicated elements for the teams, listed together so we need to drop every other one 

In [1319]:
def altElement(a):
    return a[::2]
#Get every other element in list bc teams are listed twice each

In [1320]:
away_teams = altElement(away_teams)
home_teams = altElement(home_teams)

In [1321]:
away_teams = [i.split('Lineup')[0] for i in away_teams]
home_teams = [i.split('Lineup')[0] for i in home_teams]

In [1322]:
away_teams = [i.split(' ') [-2] for i in away_teams]
home_teams = [i.split(' ') [-2] for i in home_teams]
#Get second to last element in this list which is the team name

Now I need to get the player names in the lineups

In [1323]:
home_lineups

#Messing up bc there isn't a comma till after everyone in certain team so its taking the [0] player of every team

['<ol class="starting-lineups__team starting-lineups__team--away">\n<li class="starting-lineups__player"><a class="starting-lineups__player--link" href="/player/nicky-lopez-670032" target="">Nicky Lopez</a><span class="starting-lineups__player--position"> (L) 2B</span></li>\n<li class="starting-lineups__player"><a class="starting-lineups__player--link" href="/player/bobby-witt-jr-677951" target="">Bobby Witt Jr.</a><span class="starting-lineups__player--position"> (R) SS</span></li>\n<li class="starting-lineups__player"><a class="starting-lineups__player--link" href="/player/vinnie-pasquantino-686469" target="">Vinnie Pasquantino</a><span class="starting-lineups__player--position"> (L) DH</span></li>\n<li class="starting-lineups__player"><a class="starting-lineups__player--link" href="/player/edward-olivares-658668" target="">Edward Olivares</a><span class="starting-lineups__player--position"> (R) LF</span></li>\n<li class="starting-lineups__player"><a class="starting-lineups__player--

In [1324]:
home_lineups = [i.split('target=""') for i in home_lineups]
# home_lineups = [i.split('</a><span') for i in home_lineups]
# home_lineups

In [1325]:
flat_list = []
# iterating over the data
for item in home_lineups:
    # appending elements to the flat_list
    flat_list += item

In [1326]:
flat_list

['<ol class="starting-lineups__team starting-lineups__team--away">\n<li class="starting-lineups__player"><a class="starting-lineups__player--link" href="/player/nicky-lopez-670032" ',
 '>Nicky Lopez</a><span class="starting-lineups__player--position"> (L) 2B</span></li>\n<li class="starting-lineups__player"><a class="starting-lineups__player--link" href="/player/bobby-witt-jr-677951" ',
 '>Bobby Witt Jr.</a><span class="starting-lineups__player--position"> (R) SS</span></li>\n<li class="starting-lineups__player"><a class="starting-lineups__player--link" href="/player/vinnie-pasquantino-686469" ',
 '>Vinnie Pasquantino</a><span class="starting-lineups__player--position"> (L) DH</span></li>\n<li class="starting-lineups__player"><a class="starting-lineups__player--link" href="/player/edward-olivares-658668" ',
 '>Edward Olivares</a><span class="starting-lineups__player--position"> (R) LF</span></li>\n<li class="starting-lineups__player"><a class="starting-lineups__player--link" href="/pla

In [1327]:
flat_list = [i.split('</a>')[0] for i in flat_list]
substring = '<ol'

In [1328]:
for i in range(len(flat_list)):
    if 'TBD' in flat_list[i]:
        flat_list[i] = '>TBD'
#The teams who have not been announced, whose games got postponed etc. are represented by one line that includes 'TBD' so we
# will isolate the TBD here

In [1329]:
flat_list = [x for x in flat_list if "player--" not in x]

In [1330]:
given_value ='>TBD'
new_list=[]
for i in range(len(flat_list)):
    if 'TBD' in flat_list[i]:
        flat_list.extend(repeat(given_value,8))
#Give TBD value for all 9 players plus 9 more because we will be removing every other 9 elements due to HTML structure

In [1331]:
flat_list = [i[1:] for i in flat_list]
#Remove extra > that all of the elements currently have 

In [1332]:
flat_list
#ee

['Nicky Lopez',
 'Bobby Witt Jr.',
 'Vinnie Pasquantino',
 'Edward Olivares',
 "Ryan O'Hearn",
 'Nate Eaton',
 'Nick Pratto',
 'Michael Massey',
 'Freddy Fermin',
 'N Lopez',
 'B Witt Jr.',
 'V Pasquantino',
 'E Olivares',
 "R O'Hearn",
 'N Eaton',
 'N Pratto',
 'M Massey',
 'F Fermin',
 'Ronald Acuna Jr.',
 'Dansby Swanson',
 'Matt Olson',
 'Austin Riley',
 'Eddie Rosario',
 'Adam Duvall',
 'William Contreras',
 'Robinson Cano',
 'Michael Harris II',
 'R Acuña',
 'D Swanson',
 'M Olson',
 'A Riley',
 'E Rosario',
 'A Duvall',
 'W Contreras',
 'R Canó',
 'M Harris II',
 'Jarren Duran',
 'Rafael Devers',
 'Xander Bogaerts',
 'J.D. Martinez',
 'Alex Verdugo',
 'Christian Vazquez',
 'Franchy Cordero',
 'Jackie Bradley Jr.',
 'Jeter Downs',
 'J Duran',
 'R Devers',
 'X Bogaerts',
 'J Martinez',
 'A Verdugo',
 'C Vázquez',
 'F Cordero',
 'J Bradley',
 'J Downs',
 'Cedric Mullins',
 'Trey Mancini',
 'Anthony Santander',
 'Ryan Mountcastle',
 'Austin Hays',
 'Adley Rutschman',
 'Ramon Urias',

# What to do/What do I want to find out?

1. Group Pitchers by top 4 pitches and order them by pitch_usage in descending order


In [1333]:
batter_run_value.sort_values(by=['last_name'])

,last_name,first_name,player_id,team_name_alt,pitch_type,pitch_name,run_value_per_100,run_value,pitches,pitch_usage,...,ba,slg,woba,whiff_percent,k_percent,put_away,est_ba,est_slg,est_woba,hard_hit_percent
2198,Abrams,C.J.,682928,SD,CUKC,Curveball,2.5,1,41,11.1,...,0.222,0.333,0.296,28.6,8.3,10.0,0.269,0.384,0.409,22.2
849,Abrams,C.J.,682928,SD,FF,4-Seamer,-0.5,-1,140,37.7,...,0.313,0.469,0.358,22.5,16.2,14.3,0.245,0.431,0.311,25.0
1578,Abrams,C.J.,682928,SD,CH,Changeup,-7.9,-3,41,11.1,...,0.190,0.190,0.171,24.1,19.0,36.4,0.267,0.308,0.250,29.4
1662,Abrams,C.J.,682928,SD,SL,Slider,-5.0,-3,58,15.6,...,0.056,0.056,0.084,34.3,26.3,33.3,0.126,0.190,0.162,30.8
1183,Abreu,José,547989,CWS,CUKC,Curveball,3.2,4,140,9.6,...,0.346,0.654,0.443,34.7,32.1,18.0,0.246,0.527,0.348,58.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820,d'Arnaud,Travis,518595,ATL,SIFT,Sinker,3.3,5,154,17.6,...,0.343,0.657,0.445,17.1,7.9,8.8,0.399,0.803,0.517,40.6
263,d'Arnaud,Travis,518595,ATL,SL,Slider,-1.6,-4,245,28.0,...,0.167,0.273,0.224,30.2,26.8,21.1,0.170,0.245,0.213,27.7
2102,d'Arnaud,Travis,518595,ATL,CUKC,Curveball,6.9,4,54,6.2,...,0.462,0.846,0.554,35.5,23.1,15.8,0.422,0.680,0.466,50.0
317,d'Arnaud,Travis,518595,ATL,FF,4-Seamer,0.4,1,274,31.3,...,0.274,0.532,0.367,18.4,17.9,14.3,0.245,0.496,0.337,52.0


In [1334]:
pitcher_run_value.sort_values(by=['last_name'])

,last_name,first_name,player_id,team_name_alt,pitch_type,pitch_name,run_value_per_100,run_value,pitches,pitch_usage,...,ba,slg,woba,whiff_percent,k_percent,put_away,est_ba,est_slg,est_woba,hard_hit_percent
1055,Abreu,Albert,656061,NYY,SIFT,Sinker,-1.7,-2,124,35.0,...,0.200,0.240,0.321,19.1,16.1,25.0,0.163,0.215,0.266,45.0
1792,Abreu,Albert,656061,NYY,CH,Changeup,2.0,1,40,11.3,...,0.200,0.800,0.427,42.9,27.3,30.0,0.339,1.024,0.565,42.9
349,Abreu,Bryan,650556,HOU,SL,Slider,-1.5,-4,259,49.0,...,0.179,0.224,0.245,46.5,42.7,31.4,0.186,0.221,0.233,20.0
1401,Abreu,Albert,656061,NYY,SL,Slider,-1.3,-1,104,29.4,...,0.167,0.222,0.223,43.9,40.0,23.5,0.145,0.173,0.194,0.0
626,Abreu,Bryan,650556,HOU,FF,4-Seamer,0.4,1,254,48.0,...,0.268,0.366,0.346,25.6,19.6,17.9,0.271,0.495,0.416,40.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216,Zimmermann,Bruce,669145,BAL,SIFT,Sinker,4.0,3,86,8.4,...,0.360,0.920,0.562,6.7,4.0,16.7,0.366,0.828,0.492,41.7
179,Zimmermann,Bruce,669145,BAL,CH,Changeup,2.0,6,305,29.7,...,0.316,0.561,0.400,18.9,14.7,15.8,0.285,0.530,0.352,35.7
1019,Zimmermann,Bruce,669145,BAL,CUKC,Curveball,-2.6,-3,130,12.7,...,0.161,0.290,0.208,23.0,25.0,20.0,0.251,0.451,0.308,39.1
1020,Álvarez,José,501625,SF,CH,Changeup,1.5,1,96,35.2,...,0.207,0.517,0.364,37.7,28.1,29.0,0.281,0.571,0.385,25.0
